In [75]:
!pip install findspark


Defaulting to user installation because normal site-packages is not writeable


In [76]:

import findspark
findspark.init()


# Start PySpark Session

In [77]:

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("MySparkApp").getOrCreate()


24/02/15 18:58:20 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


# Load the dataset into a DataFrame

In [4]:
data_path = "purchase.csv"
purchase_df = spark.read.csv(data_path, header=True, inferSchema=True)

# Display the schema and first few rows of the DataFrame
purchase_df.printSchema()
purchase_df.show(5)


root
 |-- event_time: timestamp (nullable = true)
 |-- order_id: long (nullable = true)
 |-- product_id: long (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: long (nullable = true)

+-------------------+-------------------+-------------------+-------------------+--------------------+-------+------+-------------------+
|         event_time|           order_id|         product_id|        category_id|       category_code|  brand| price|            user_id|
+-------------------+-------------------+-------------------+-------------------+--------------------+-------+------+-------------------+
|2020-04-24 17:20:39|2294359932054536986|1515966223509089906|2268105426648170900|  electronics.tablet|samsung|162.01|1515915625441993984|
|2020-04-24 17:20:39|2294359932054536986|1515966223509089906|2268105426648170900|  electronics.tablet|samsung|162.01|15159

In [17]:
row_count = purchase_df.count()

print(row_count)


Number of rows: 2633521


# Drop dulplicate rows

In [18]:
purchase_df = purchase_df.dropDuplicates()

# Count number of rows

In [19]:
row_count = purchase_df.count()

print(row_count)

24/02/13 18:03:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/02/13 18:03:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/02/13 18:03:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/02/13 18:03:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/02/13 18:03:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/02/13 18:03:20 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/02/13 18:03:21 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/02/13 18:03:24 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/02/13 18:03:24 WARN RowBasedKeyValueBatch: Calling spill() on

Number of rows: 2632846


# Datatype of column

In [20]:
purchase_df.dtypes

[('event_time', 'timestamp'),
 ('order_id', 'bigint'),
 ('product_id', 'bigint'),
 ('category_id', 'bigint'),
 ('category_code', 'string'),
 ('brand', 'string'),
 ('price', 'double'),
 ('user_id', 'bigint')]

In [21]:
print(purchase_df)

DataFrame[event_time: timestamp, order_id: bigint, product_id: bigint, category_id: bigint, category_code: string, brand: string, price: double, user_id: bigint]


# Filter out rows where price is not null

In [25]:
filtered_df = purchase_df.filter(purchase_df["price"].isNotNull())
filtered_df.show()


24/02/13 18:21:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+-------------------+-------------------+-------------------+-------------------+--------------------+-------+------+-------------------+
|         event_time|           order_id|         product_id|        category_id|       category_code|  brand| price|            user_id|
+-------------------+-------------------+-------------------+-------------------+--------------------+-------+------+-------------------+
|2020-04-29 20:11:49|2298069964415828136|1515966223509122874|2268105407933187062|computers.periphe...|     hp|152.52|1515915625443027224|
|2020-04-29 23:42:11|2298175846491357353|1515966223509122666|2268105430162997728|electronics.audio...|samsung|  8.08|1515915625445938216|
|2020-04-30 18:01:51|2298729326712980173|1515966223509089265|2360741866917331945|appliances.enviro...|   beko|231.46|1515915625446617606|
|2020-04-30 19:27:36|2298772487720140990|1515966223509335414|2268105430162997728|electronics.audio...|  razer|104.14|1515915625452727322|
|2020-04-30 21:37:48|2298838016631

# Rows after deletion

In [26]:
rc = filtered_df.count()

print(rc)

24/02/13 18:22:14 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/02/13 18:22:15 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


2200893


# Filter out rows where brand is not null

In [29]:
filtered_df = filtered_df.filter(filtered_df["brand"].isNotNull())

filtered_df.show()


+-------------------+-------------------+-------------------+-------------------+--------------------+-------+------+-------------------+
|         event_time|           order_id|         product_id|        category_id|       category_code|  brand| price|            user_id|
+-------------------+-------------------+-------------------+-------------------+--------------------+-------+------+-------------------+
|2020-04-29 20:11:49|2298069964415828136|1515966223509122874|2268105407933187062|computers.periphe...|     hp|152.52|1515915625443027224|
|2020-04-29 23:42:11|2298175846491357353|1515966223509122666|2268105430162997728|electronics.audio...|samsung|  8.08|1515915625445938216|
|2020-04-30 18:01:51|2298729326712980173|1515966223509089265|2360741866917331945|appliances.enviro...|   beko|231.46|1515915625446617606|
|2020-04-30 19:27:36|2298772487720140990|1515966223509335414|2268105430162997728|electronics.audio...|  razer|104.14|1515915625452727322|
|2020-04-30 21:37:48|2298838016631

# Rows after deletion

In [30]:
rc = filtered_df.count()

print(rc)

2088263


# Inconsistent Records

In [31]:
from pyspark.sql.functions import col, countDistinct

# Group by 'order_id' and count the distinct 'event_time' values
grouped_df = filtered_df.groupBy('order_id').agg(countDistinct('event_time').alias('num_distinct_event_time'))

# Filter for inconsistent order ids where the number of distinct event times is greater than 1
inconsistent_order_ids = grouped_df.filter(col('num_distinct_event_time') > 1).select('order_id')

# Join with filtered_df to get the records with inconsistent event_time
inconsistent_records = filtered_df.join(inconsistent_order_ids, 'order_id', 'inner')

# Count the number of inconsistent records
num_inconsistent_records = inconsistent_records.count()

print("Number of records with inconsistent event_time:", num_inconsistent_records)



Number of records with inconsistent event_time: 0


# Unique User IDs

In [34]:
df = filtered_df# Count the number of unique user_id
unique_user_count = df.select("user_id").distinct().count()

# Display the count of unique user_id
print(unique_user_count)


Number of unique user_id: 229229


# Unique Order IDs

In [35]:
unique_order_count = df.select("order_id").distinct().count()

# Display the count of unique user_id
print(unique_order_count)

Number of unique user_id: 1376122


# Inconsistent order ID

In [36]:
from pyspark.sql import functions as F

# Step 1: Identify duplicate order_id entries
duplicate_orders_df = df.groupBy("order_id").count().where(F.col("count") > 1)

# Step 2: Group by order_id and count distinct user_id values
distinct_user_count_df = df.groupBy("order_id").agg(F.countDistinct("user_id").alias("distinct_user_count"))

# Step 3: Filter out groups where there is more than one distinct user_id
filtered_orders_df = distinct_user_count_df.join(duplicate_orders_df, "order_id", "inner").filter("distinct_user_count > 1")

# Step 4: Count the total number of rows in the filtered DataFrame
total_rows_with_different_user_id = filtered_orders_df.select(F.sum("count")).collect()[0][0]

# Display the result
print("Total number of rows where each duplicate order_id does not have the same user_id:", total_rows_with_different_user_id)


Total number of rows where each duplicate order_id does not have the same user_id: None


In [37]:
df.show()

+-------------------+-------------------+-------------------+-------------------+--------------------+-------+------+-------------------+
|         event_time|           order_id|         product_id|        category_id|       category_code|  brand| price|            user_id|
+-------------------+-------------------+-------------------+-------------------+--------------------+-------+------+-------------------+
|2020-04-29 20:11:49|2298069964415828136|1515966223509122874|2268105407933187062|computers.periphe...|     hp|152.52|1515915625443027224|
|2020-04-29 23:42:11|2298175846491357353|1515966223509122666|2268105430162997728|electronics.audio...|samsung|  8.08|1515915625445938216|
|2020-04-30 18:01:51|2298729326712980173|1515966223509089265|2360741866917331945|appliances.enviro...|   beko|231.46|1515915625446617606|
|2020-04-30 19:27:36|2298772487720140990|1515966223509335414|2268105430162997728|electronics.audio...|  razer|104.14|1515915625452727322|
|2020-04-30 21:37:48|2298838016631

# Calculate the number of null values for each column


In [40]:
null_counts = [F.sum(F.col(column).isNull().cast("integer")).alias(column) for column in df.columns]

# Describe the DataFrame with null value counts
null_counts_df = df.agg(*null_counts)

null_counts_df.show()


+----------+--------+----------+-----------+-------------+-----+-----+-------+
|event_time|order_id|product_id|category_id|category_code|brand|price|user_id|
+----------+--------+----------+-----------+-------------+-----+-----+-------+
|         0|       0|         0|          0|       556588|    0|    0|1551952|
+----------+--------+----------+-----------+-------------+-----+-----+-------+



# Calculate the number of null values in the "category_id" column


In [41]:
from pyspark.sql.functions import col, sum

null_category_id_count = df.select(sum(col("category_id").isNull().cast("int"))).collect()[0][0]

# Display the number of null values in the "category_id" column
print("Number of null values in the 'category_id' column:", null_category_id_count)


Number of null values in the 'category_id' column: 0


In [42]:
df.dtypes

[('event_time', 'timestamp'),
 ('order_id', 'bigint'),
 ('product_id', 'bigint'),
 ('category_id', 'bigint'),
 ('category_code', 'string'),
 ('brand', 'string'),
 ('price', 'double'),
 ('user_id', 'bigint')]

In [43]:
rc = df.count()

print(rc)

2088263


# Filter out rows where category_code is not null


In [44]:
fdf = df.filter(df["category_code"].isNotNull())

fdf.show()


+-------------------+-------------------+-------------------+-------------------+--------------------+-------+------+-------------------+
|         event_time|           order_id|         product_id|        category_id|       category_code|  brand| price|            user_id|
+-------------------+-------------------+-------------------+-------------------+--------------------+-------+------+-------------------+
|2020-04-29 20:11:49|2298069964415828136|1515966223509122874|2268105407933187062|computers.periphe...|     hp|152.52|1515915625443027224|
|2020-04-29 23:42:11|2298175846491357353|1515966223509122666|2268105430162997728|electronics.audio...|samsung|  8.08|1515915625445938216|
|2020-04-30 18:01:51|2298729326712980173|1515966223509089265|2360741866917331945|appliances.enviro...|   beko|231.46|1515915625446617606|
|2020-04-30 19:27:36|2298772487720140990|1515966223509335414|2268105430162997728|electronics.audio...|  razer|104.14|1515915625452727322|
|2020-04-30 21:37:48|2298838016631

In [46]:
rc = fdf.count()

print(rc)

1531675


# Count unique records in category_code

In [52]:
from pyspark.sql.functions import countDistinct

distinct_category_code_count = df.select(countDistinct("category_code")).collect()[0][0]

print(distinct_category_code_count)


123


# count the distinct records before the first dot '.' 

In [49]:
from pyspark.sql.functions import split, col

# Extract the category before the first dot '.'
df_with_category = fdf.withColumn("category_before_dot", split(col("category_code"), "\\.")[0])

# Count the distinct records before the first dot '.'
distinct_category_before_dot_count = df_with_category.select("category_before_dot").distinct().count()

# Display the count of distinct records before the first dot '.'
print("Number of distinct records before the first dot '.':", distinct_category_before_dot_count)


Number of distinct records before the first dot '.': 13


# Display the distinct names of records before the first dot '.'


In [50]:
from pyspark.sql.functions import split, col

# Extract the category before the first dot '.'
df_with_category = fdf.withColumn("category_before_dot", split(col("category_code"), "\\.")[0])

# Select and display the distinct names of records before the first dot '.'
distinct_category_before_dot_names = df_with_category.select("category_before_dot").distinct().collect()
distinct_category_before_dot_names = [row["category_before_dot"] for row in distinct_category_before_dot_names]

print("Distinct names of records before the first dot '.' in the 'category_code' column:")
for name in distinct_category_before_dot_names:
    print(name)


Distinct names of records before the first dot '.' in the 'category_code' column:
medicine
computers
auto
stationery
sport
apparel
appliances
country_yard
furniture
accessories
kids
electronics
construction


# Count Records that start with 'string'

In [53]:
from pyspark.sql.functions import col

category_code_count = df.filter(col("category_code").startswith('medicine')).count()

print(category_code_count)


3369


In [54]:
from pyspark.sql.functions import col

category_code_count = df.filter(col("category_code").startswith('computers')).count()

print(category_code_count)


209801


In [55]:
from pyspark.sql.functions import col

category_code_count = df.filter(col("category_code").startswith('auto')).count()

print(category_code_count)


3258


In [56]:
from pyspark.sql.functions import col

category_code_count = df.filter(col("category_code").startswith('stationery')).count()

print(category_code_count)


38924


In [57]:
from pyspark.sql.functions import col

category_code_count = df.filter(col("category_code").startswith('sport')).count()

print(category_code_count)


1803


In [58]:
from pyspark.sql.functions import col

category_code_count = df.filter(col("category_code").startswith('apparel')).count()

print(category_code_count)


7707


In [59]:
from pyspark.sql.functions import col

category_code_count = df.filter(col("category_code").startswith('appliances')).count()

print(category_code_count)


589568


In [60]:
from pyspark.sql.functions import col

category_code_count = df.filter(col("category_code").startswith('country_yard')).count()

print(category_code_count)


303


In [61]:
from pyspark.sql.functions import col

category_code_count = df.filter(col("category_code").startswith('furniture')).count()

print(category_code_count)


104568


In [62]:
from pyspark.sql.functions import col

category_code_count = df.filter(col("category_code").startswith('accessories')).count()

print(category_code_count)


13401


In [63]:
from pyspark.sql.functions import col

category_code_count = df.filter(col("category_code").startswith('kids')).count()

print(category_code_count)


4969


In [64]:
from pyspark.sql.functions import col

category_code_count = df.filter(col("category_code").startswith('electronics')).count()

print(category_code_count)


541435


In [66]:
from pyspark.sql.functions import col

category_code_count = df.filter(col("category_code").startswith('construction')).count()

print(category_code_count)


12569


# Download clean data file

In [75]:
# Specify the path where you want to save the cleaned CSV file
output_path = "clean_file"

# Write the cleaned DataFrame to a single CSV file
df.coalesce(1).write.csv(output_path, header=True)


# Download cleaned file w/o any null records in category_code

In [76]:
# Specify the path where you want to save the cleaned CSV file
output_path = "filered_clean_file"

# Write the cleaned DataFrame to a single CSV file
fdf.coalesce(1).write.csv(output_path, header=True)


In [6]:
data_path = "filtered_clean.csv"
fdf1 = spark.read.csv(data_path, header=True, inferSchema=True)



In [13]:
rc = fdf1.count()

print(rc)

1531675


In [14]:
from pyspark.sql import functions as F
null_counts = [F.sum(F.col(column).isNull().cast("integer")).alias(column) for column in fdf1.columns]

# Describe the DataFrame with null value counts
null_counts_df = fdf1.agg(*null_counts)

null_counts_df.show()

+----------+--------+----------+-----------+-------------+-----+-----+-------+
|event_time|order_id|product_id|category_id|category_code|brand|price|user_id|
+----------+--------+----------+-----------+-------------+-----+-----+-------+
|         0|       0|         0|          0|            0|    0|    0|1111457|
+----------+--------+----------+-----------+-------------+-----+-----+-------+



# Load dataset into file

In [5]:
data_path = "clean_file.csv"
df1 = spark.read.csv(data_path, header=True, inferSchema=True)


In [81]:
rc = df.count()

print(rc)

rc = df1.count()

print(rc)

2088263
2088263


In [6]:
from pyspark.sql import functions as F
null_counts = [F.sum(F.col(column).isNull().cast("integer")).alias(column) for column in df1.columns]

# Describe the DataFrame with null value counts
null_counts_df = df1.agg(*null_counts)

null_counts_df.show()

+----------+--------+----------+-----------+-------------+-----+-----+-------+
|event_time|order_id|product_id|category_id|category_code|brand|price|user_id|
+----------+--------+----------+-----------+-------------+-----+-----+-------+
|         0|       0|         0|          0|       556588|    0|    0|1551952|
+----------+--------+----------+-----------+-------------+-----+-----+-------+



In [17]:
unique_order_count = fdf1.select("order_id").distinct().count()

# Display the count of unique user_id
print(unique_order_count)

1143398


In [18]:
unique_order_count = fdf1.select("user_id").distinct().count()

# Display the count of unique user_id
print(unique_order_count)

203479


In [19]:
fdf1.dtypes

[('event_time', 'timestamp'),
 ('order_id', 'bigint'),
 ('product_id', 'bigint'),
 ('category_id', 'bigint'),
 ('category_code', 'string'),
 ('brand', 'string'),
 ('price', 'double'),
 ('user_id', 'bigint')]

In [21]:
unique_order_count = fdf1.select("product_id").distinct().count()

# Display the count of unique user_id
print(unique_order_count)

13169


In [22]:
unique_order_count = fdf1.select("category_id").distinct().count()

# Display the count of unique user_id
print(unique_order_count)

359


In [23]:
# from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window


# Define parameters
start_user_id = 911562541901000
num_unique_user_ids = 100000

# Generate random user_id1 values
random_user_ids = spark.range(start_user_id, start_user_id + num_unique_user_ids).select(F.col("id").cast("bigint").alias("user_id1"))

# Create a DataFrame named df_cleaned (assuming it's already available)
# Assuming df_cleaned has columns 'order_id' and 'user_id1'

# Group by 'order_id' and get distinct values
unique_order_ids = fdf1.select('order_id').distinct()

# Assign random user_id1 to each unique order_id
windowSpec = Window.orderBy(F.monotonically_increasing_id())
random_user_ids = random_user_ids.withColumn("row_num", F.row_number().over(windowSpec))

# Join with unique order_ids
mapped_user_ids = unique_order_ids.withColumn("row_num", F.monotonically_increasing_id()).join(random_user_ids, on="row_num", how="inner").drop("row_num")

# Map order_id to user_id1
fdf2 = fdf1.join(mapped_user_ids, on="order_id", how="left")

# Show DataFrame
print("DataFrame with new column user_id1:")
fdf2.show()




DataFrame with new column user_id1:


24/02/13 23:24:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/02/13 23:24:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/02/13 23:24:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/02/13 23:24:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/02/13 23:24:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/02/13 23:24:11 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------------------+-------------------+-------------------+-------------------+--------------------+---------+------+-------------------+---------------+
|           order_id|         event_time|         product_id|        category_id|       category_code|    brand| price|            user_id|       user_id1|
+-------------------+-------------------+-------------------+-------------------+--------------------+---------+------+-------------------+---------------+
|2298069964415828136|2020-04-29 20:11:49|1515966223509122874|2268105407933187062|computers.periphe...|       hp|152.52|1515915625443027224|911562541932228|
|2300342008322982139|2020-05-02 23:25:58|2273948248047616169|2268105406549066706|computers.periphe...|    delux|  5.07|1515915625444068089|911562541947545|
|2302886286089781416|2020-05-06 11:40:59|1515966223509089722|2268105441856717530|appliances.kitche...|      ava|  9.24|1515915625455413662|911562541942454|
|2303774668819006287|2020-05-07 17:06:03|1515966223509104779|226

In [25]:
from pyspark.sql import functions as F
null_counts = [F.sum(F.col(column).isNull().cast("integer")).alias(column) for column in fdf2.columns]

# Describe the DataFrame with null value counts
null_counts_df = fdf2.agg(*null_counts)

null_counts_df.show()

24/02/13 23:25:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/02/13 23:25:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/02/13 23:25:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/02/13 23:25:59 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/02/13 23:26:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/02/13 23:26:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+--------+----------+----------+-----------+-------------+-----+-----+-------+--------+
|order_id|event_time|product_id|category_id|category_code|brand|price|user_id|user_id1|
+--------+----------+----------+-----------+-------------+-----+-----+-------+--------+
|       0|         0|         0|          0|            0|    0|    0|1111457| 1388443|
+--------+----------+----------+-----------+-------------+-----+-----+-------+--------+



# Unique event_time 

In [11]:
# Count the distinct values in the "event_time" column
distinct_event_time_count = fdf1.select("event_time").distinct().count()

# Display the count of distinct values
print("Number of distinct values in the 'event_time' column:", distinct_event_time_count)


Number of distinct values in the 'event_time' column: 1064860


In [12]:
# Count the distinct values in the "event_time" column
distinct_event_time_count = fdf1.select("order_id").distinct().count()

# Display the count of distinct values
print("Number of distinct values in the 'order_id' column:", distinct_event_time_count)


Number of distinct values in the 'order_id' column: 1143398


In [6]:
rc = df1.count()
print(rc)

2088263


In [7]:
df1.show()

+-------------------+-------------------+-------------------+-------------------+--------------------+-------+------+-------------------+
|         event_time|           order_id|         product_id|        category_id|       category_code|  brand| price|            user_id|
+-------------------+-------------------+-------------------+-------------------+--------------------+-------+------+-------------------+
|2020-04-29 20:11:49|2298069964415828136|1515966223509122874|2268105407933187062|computers.periphe...|     hp|152.52|1515915625443027224|
|2020-04-29 23:42:11|2298175846491357353|1515966223509122666|2268105430162997728|electronics.audio...|samsung|  8.08|1515915625445938216|
|2020-04-30 18:01:51|2298729326712980173|1515966223509089265|2360741866917331945|appliances.enviro...|   beko|231.46|1515915625446617606|
|2020-04-30 19:27:36|2298772487720140990|1515966223509335414|2268105430162997728|electronics.audio...|  razer|104.14|1515915625452727322|
|2020-04-30 21:37:48|2298838016631

# Count null values in dataset

In [9]:
from pyspark.sql import functions as F
null_counts = [F.sum(F.col(column).isNull().cast("integer")).alias(column) for column in df1.columns]

# Describe the DataFrame with null value counts
null_counts_df = df1.agg(*null_counts)

null_counts_df.show()

+----------+--------+----------+-----------+-------------+-----+-----+-------+
|event_time|order_id|product_id|category_id|category_code|brand|price|user_id|
+----------+--------+----------+-----------+-------------+-----+-----+-------+
|         0|       0|         0|          0|       556588|    0|    0|1551952|
+----------+--------+----------+-----------+-------------+-----+-----+-------+



# Assign 'other' in category_code where values are null

In [10]:
from pyspark.sql.functions import when

# Assign 'other' to null values in the "category_code" column
odf = df1.withColumn("category_code", when(df1["category_code"].isNull(), "other").otherwise(df1["category_code"]))

# Show the updated DataFrame
odf.show()


+-------------------+-------------------+-------------------+-------------------+--------------------+-------+------+-------------------+
|         event_time|           order_id|         product_id|        category_id|       category_code|  brand| price|            user_id|
+-------------------+-------------------+-------------------+-------------------+--------------------+-------+------+-------------------+
|2020-04-29 20:11:49|2298069964415828136|1515966223509122874|2268105407933187062|computers.periphe...|     hp|152.52|1515915625443027224|
|2020-04-29 23:42:11|2298175846491357353|1515966223509122666|2268105430162997728|electronics.audio...|samsung|  8.08|1515915625445938216|
|2020-04-30 18:01:51|2298729326712980173|1515966223509089265|2360741866917331945|appliances.enviro...|   beko|231.46|1515915625446617606|
|2020-04-30 19:27:36|2298772487720140990|1515966223509335414|2268105430162997728|electronics.audio...|  razer|104.14|1515915625452727322|
|2020-04-30 21:37:48|2298838016631

# After assigning again check null values in dataset

In [11]:
from pyspark.sql import functions as F
null_counts = [F.sum(F.col(column).isNull().cast("integer")).alias(column) for column in odf.columns]

# Describe the DataFrame with null value counts
null_counts_df = odf.agg(*null_counts)

null_counts_df.show()

+----------+--------+----------+-----------+-------------+-----+-----+-------+
|event_time|order_id|product_id|category_id|category_code|brand|price|user_id|
+----------+--------+----------+-----------+-------------+-----+-----+-------+
|         0|       0|         0|          0|            0|    0|    0|1551952|
+----------+--------+----------+-----------+-------------+-----+-----+-------+



# Download file 

In [12]:
output_path = "other_clean_file"

odf.coalesce(1).write.csv(output_path, header=True)


# Load file 

In [5]:
data_path = "other_clean.csv"
odf = spark.read.csv(data_path, header=True, inferSchema=True)


# Add new column name 'user_id1'

In [6]:
cdf = odf.withColumn("user_id1", odf["user_id"])

cdf.show()


+-------------------+-------------------+-------------------+-------------------+--------------------+-------+------+-------------------+-------------------+
|         event_time|           order_id|         product_id|        category_id|       category_code|  brand| price|            user_id|           user_id1|
+-------------------+-------------------+-------------------+-------------------+--------------------+-------+------+-------------------+-------------------+
|2020-04-29 20:11:49|2298069964415828136|1515966223509122874|2268105407933187062|computers.periphe...|     hp|152.52|1515915625443027224|1515915625443027224|
|2020-04-29 23:42:11|2298175846491357353|1515966223509122666|2268105430162997728|electronics.audio...|samsung|  8.08|1515915625445938216|1515915625445938216|
|2020-04-30 18:01:51|2298729326712980173|1515966223509089265|2360741866917331945|appliances.enviro...|   beko|231.46|1515915625446617606|1515915625446617606|
|2020-04-30 19:27:36|2298772487720140990|15159662235

In [7]:
from pyspark.sql.functions import lit

cdf1 = odf.withColumn("user_id1", lit(None).cast("bigint"))

cdf1.show()


+-------------------+-------------------+-------------------+-------------------+--------------------+-------+------+-------------------+--------+
|         event_time|           order_id|         product_id|        category_id|       category_code|  brand| price|            user_id|user_id1|
+-------------------+-------------------+-------------------+-------------------+--------------------+-------+------+-------------------+--------+
|2020-04-29 20:11:49|2298069964415828136|1515966223509122874|2268105407933187062|computers.periphe...|     hp|152.52|1515915625443027224|    NULL|
|2020-04-29 23:42:11|2298175846491357353|1515966223509122666|2268105430162997728|electronics.audio...|samsung|  8.08|1515915625445938216|    NULL|
|2020-04-30 18:01:51|2298729326712980173|1515966223509089265|2360741866917331945|appliances.enviro...|   beko|231.46|1515915625446617606|    NULL|
|2020-04-30 19:27:36|2298772487720140990|1515966223509335414|2268105430162997728|electronics.audio...|  razer|104.14|1

# Download cdf1 file

In [9]:
output_path = "cdf1_file"

cdf1.coalesce(1).write.csv(output_path, header=True)


# Load cdf1 file

In [5]:
data_path = "cdf1_file.csv"
cdf1 = spark.read.csv(data_path, header=True, inferSchema=True)


In [20]:
# from pyspark.sql import SparkSession
from pyspark.sql.functions import col, rand

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("AssignUserIDs") \
    .getOrCreate()

# Sample data - replace this with your actual DataFrame
# df_cleaned = spark.createDataFrame([...], ["order_id", ...])

# Generate a DataFrame with unique order_id and a random user_id1
unique_order_ids = cdf1.select("order_id").distinct()
unique_order_ids = unique_order_ids.withColumn("user_id3", (rand() * (911562561901000 - 911562541901000) + 911562541901000).cast("bigint"))

# Join the original DataFrame with the DataFrame containing unique order_id and user_id1
udf = cdf1.join(unique_order_ids, "order_id", "left")

# Show the DataFrame
udf.show()

# Stop SparkSession
# spark.stop()


+-------------------+-------------------+-------------------+-------------------+--------------------+---------+------+-------------------+--------+---------------+
|           order_id|         event_time|         product_id|        category_id|       category_code|    brand| price|            user_id|user_id1|       user_id3|
+-------------------+-------------------+-------------------+-------------------+--------------------+---------+------+-------------------+--------+---------------+
|2298069964415828136|2020-04-29 20:11:49|1515966223509122874|2268105407933187062|computers.periphe...|       hp|152.52|1515915625443027224|    NULL|911562558747193|
|2300342008322982139|2020-05-02 23:25:58|2273948248047616169|2268105406549066706|computers.periphe...|    delux|  5.07|1515915625444068089|    NULL|911562560279907|
|2302886286089781416|2020-05-06 11:40:59|1515966223509089722|2268105441856717530|appliances.kitche...|      ava|  9.24|1515915625455413662|    NULL|911562552616904|
|230377466

In [21]:
output_path = "Userdf_file"

udf.coalesce(1).write.csv(output_path, header=True)


In [22]:
# Count the distinct values in the "event_time" column
distinct_event_time_count = udf.select("user_id3").distinct().count()

# Display the count of distinct values
print("Number of distinct values in the 'user_id3' column:", distinct_event_time_count)


Number of distinct values in the 'user_id3' column: 1329754


In [23]:
# Count the distinct values in the "event_time" column
distinct_event_time_count = udf.select("order_id").distinct().count()

# Display the count of distinct values
print("Number of distinct values in the 'order_id' column:", distinct_event_time_count)


Number of distinct values in the 'order_id' column: 1376122


# Create new user_id column

In [24]:
from pyspark.sql.functions import col, rand

spark = SparkSession.builder \
    .appName("AssignUserIDs") \
    .getOrCreate()


# Generate a DataFrame with unique order_id and a random user_id1
unique_order_ids = cdf1.select("order_id").distinct()
unique_order_ids = unique_order_ids.withColumn("user_id4", (rand() * (50000) + 911562541901000).cast("bigint"))

# Join the original DataFrame with the DataFrame containing unique order_id and user_id1
udf1 = cdf1.join(unique_order_ids, "order_id", "left")

udf1.show()



+-------------------+-------------------+-------------------+-------------------+--------------------+---------+------+-------------------+--------+---------------+
|           order_id|         event_time|         product_id|        category_id|       category_code|    brand| price|            user_id|user_id1|       user_id4|
+-------------------+-------------------+-------------------+-------------------+--------------------+---------+------+-------------------+--------+---------------+
|2298069964415828136|2020-04-29 20:11:49|1515966223509122874|2268105407933187062|computers.periphe...|       hp|152.52|1515915625443027224|    NULL|911562541902984|
|2300342008322982139|2020-05-02 23:25:58|2273948248047616169|2268105406549066706|computers.periphe...|    delux|  5.07|1515915625444068089|    NULL|911562541938535|
|2302886286089781416|2020-05-06 11:40:59|1515966223509089722|2268105441856717530|appliances.kitche...|      ava|  9.24|1515915625455413662|    NULL|911562541920980|
|230377466

In [25]:
# Count the distinct values in the "event_time" column
distinct_event_time_count = udf1.select("order_id").distinct().count()

# Display the count of distinct values
print("Number of distinct values in the 'order_id' column:", distinct_event_time_count)


Number of distinct values in the 'order_id' column: 1376122


In [26]:
# Count the distinct values in the "event_time" column
distinct_event_time_count = udf1.select("user_id4").distinct().count()

# Display the count of distinct values
print("Number of distinct values in the 'user_id4' column:", distinct_event_time_count)


Number of distinct values in the 'user_id4' column: 50000


In [28]:
from pyspark.sql import functions as F
null_counts = [F.sum(F.col(column).isNull().cast("integer")).alias(column) for column in cdf1.columns]

# Describe the DataFrame with null value counts
null_counts_df = cdf1.agg(*null_counts)

null_counts_df.show()


+----------+--------+----------+-----------+-------------+-----+-----+-------+--------+
|event_time|order_id|product_id|category_id|category_code|brand|price|user_id|user_id1|
+----------+--------+----------+-----------+-------------+-----+-----+-------+--------+
|         0|       0|         0|          0|            0|    0|    0|1551952| 2088263|
+----------+--------+----------+-----------+-------------+-----+-----+-------+--------+



In [29]:
from pyspark.sql import functions as F
null_counts = [F.sum(F.col(column).isNull().cast("integer")).alias(column) for column in udf1.columns]

# Describe the DataFrame with null value counts
null_counts_df = udf1.agg(*null_counts)

null_counts_df.show()


24/02/14 22:32:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/02/14 22:32:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/02/14 22:32:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+--------+----------+----------+-----------+-------------+-----+-----+-------+--------+--------+
|order_id|event_time|product_id|category_id|category_code|brand|price|user_id|user_id1|user_id4|
+--------+----------+----------+-----------+-------------+-----+-----+-------+--------+--------+
|       0|         0|         0|          0|            0|    0|    0|1551952| 2088263|       0|
+--------+----------+----------+-----------+-------------+-----+-----+-------+--------+--------+



In [30]:
udf2 = udf1.drop("user_id", "user_id1")

In [31]:
from pyspark.sql import functions as F
null_counts = [F.sum(F.col(column).isNull().cast("integer")).alias(column) for column in udf2.columns]

# Describe the DataFrame with null value counts
null_counts_df = udf2.agg(*null_counts)

null_counts_df.show()


24/02/14 22:33:56 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/02/14 22:33:56 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/02/14 22:33:56 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/02/14 22:33:56 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+--------+----------+----------+-----------+-------------+-----+-----+--------+
|order_id|event_time|product_id|category_id|category_code|brand|price|user_id4|
+--------+----------+----------+-----------+-------------+-----+-----+--------+
|       0|         0|         0|          0|            0|    0|    0|       0|
+--------+----------+----------+-----------+-------------+-----+-----+--------+



In [32]:
udf3 = udf2.withColumnRenamed("user_id4", "user_id")

In [33]:
from pyspark.sql import functions as F
null_counts = [F.sum(F.col(column).isNull().cast("integer")).alias(column) for column in udf3.columns]

# Describe the DataFrame with null value counts
null_counts_df = udf3.agg(*null_counts)

null_counts_df.show()


24/02/14 22:35:10 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/02/14 22:35:10 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/02/14 22:35:10 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/02/14 22:35:10 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+--------+----------+----------+-----------+-------------+-----+-----+-------+
|order_id|event_time|product_id|category_id|category_code|brand|price|user_id|
+--------+----------+----------+-----------+-------------+-----+-----+-------+
|       0|         0|         0|          0|            0|    0|    0|      0|
+--------+----------+----------+-----------+-------------+-----+-----+-------+



In [34]:
output_path = "udf3_file"

udf3.coalesce(1).write.csv(output_path, header=True)


In [36]:
udf3.dtypes

[('order_id', 'bigint'),
 ('event_time', 'timestamp'),
 ('product_id', 'bigint'),
 ('category_id', 'bigint'),
 ('category_code', 'string'),
 ('brand', 'string'),
 ('price', 'double'),
 ('user_id', 'bigint')]

In [38]:
row_count = udf3.count()

print(row_count)


2088263


In [37]:
from pyspark.sql.functions import countDistinct

# Count the number of unique user_id values
unique_user_count = udf3.select(countDistinct("user_id")).collect()[0][0]

# Display the count of unique user_id values
print("Number of unique user_id values:", unique_user_count)


Number of unique user_id values: 50000


# Count unique Order IDs

In [39]:
from pyspark.sql.functions import countDistinct

# Count the number of unique user_id values
uc = udf3.select(countDistinct("order_id")).collect()[0][0]

# Display the count of unique user_id values
print("Number of unique order_id values:", uc)


Number of unique order_id values: 1376122


# Count unique event_time

In [40]:
from pyspark.sql.functions import countDistinct

# Count the number of unique user_id values
unique_user_count = udf3.select(countDistinct("event_time")).collect()[0][0]

# Display the count of unique user_id values
print("Number of unique event_time values:", unique_user_count)


Number of unique event_time values: 1265836


# inconsistent records where multiple duplicate order_id have multiple unique event_time

In [41]:
from pyspark.sql import functions as F

# Group by order_id and count the distinct event_time values
inconsistent_records_count = udf3.groupby("order_id").agg(F.countDistinct("event_time").alias("unique_event_times"))

# Filter for records where the count of unique event_time values is greater than 1
inconsistent_records_count = inconsistent_records_count.filter(inconsistent_records_count["unique_event_times"] > 1)

# Count the number of inconsistent records
num_inconsistent_records = inconsistent_records_count.count()

# Display the number of inconsistent records
print("Number of inconsistent records:", num_inconsistent_records)


Number of inconsistent records: 0


#  inconsistent records where multiple duplicate event_time have multiple unique order_id

In [42]:
from pyspark.sql import functions as F

# Group by event_time and count the distinct order_id values
inconsistent_records_count = udf3.groupby("event_time").agg(F.countDistinct("order_id").alias("unique_order_ids"))

# Filter for records where the count of unique order_id values is greater than 1
inconsistent_records_count = inconsistent_records_count.filter(inconsistent_records_count["unique_order_ids"] > 1)

# Count the number of inconsistent records

num_inconsistent_records_1 = inconsistent_records_count.count()

# Display the number of inconsistent records
print("Number of inconsistent records:", num_inconsistent_records_1)


Number of inconsistent records: 89455


# remove the inconsistent records from the DataFrame

In [45]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("FilterInconsistentRecords") \
    .getOrCreate()

# Sample data - replace this with your actual DataFrame
# cdf1 = spark.createDataFrame([...], ["event_time", "order_id", ...])

# Identify event_time values with more than one unique order_id
inconsistent_event_times = udf3.groupBy("event_time").agg(F.countDistinct("order_id").alias("unique_order_ids")) \
                               .where("unique_order_ids > 1") \
                               .select("event_time")

# Inner join to filter out inconsistent records
udf4 = udf3.join(inconsistent_event_times, "event_time", "left_anti")

# Show the DataFrame
udf4.show()

# Stop SparkSession
# spark.stop()


+-------------------+-------------------+-------------------+-------------------+--------------------+---------+------+---------------+
|         event_time|           order_id|         product_id|        category_id|       category_code|    brand| price|        user_id|
+-------------------+-------------------+-------------------+-------------------+--------------------+---------+------+---------------+
|2020-01-05 10:24:34|2348708941174670206|2273948316532212462|2268105428166508982|electronics.smart...|   huawei|254.61|911562541913251|
|2020-01-05 10:28:15|2348709292439241205|2273948218930758560|2268105430162997728|electronics.audio...|  samsung| 12.71|911562541934771|
|2020-01-05 10:28:15|2348709292439241205|1515966223509117074|2268105427872907696|               other|  samsung| 30.07|911562541934771|
|2020-01-05 10:28:15|2348709292439241205|1515966223509255474|2268105392070329020|appliances.enviro...|    tefal|115.72|911562541934771|
|2020-01-05 10:28:15|2348709292439241205|1515966

In [46]:
row_count = udf4.count()

print(row_count)


1778444


In [47]:
from pyspark.sql import functions as F

# Group by event_time and count the distinct order_id values
inconsistent_records_count = udf4.groupby("event_time").agg(F.countDistinct("order_id").alias("unique_order_ids"))

# Filter for records where the count of unique order_id values is greater than 1
inconsistent_records_count = inconsistent_records_count.filter(inconsistent_records_count["unique_order_ids"] > 1)

# Count the number of inconsistent records

num_inconsistent_records_1 = inconsistent_records_count.count()

# Display the number of inconsistent records
print("Number of inconsistent records:", num_inconsistent_records_1)


Number of inconsistent records: 0


In [48]:
from pyspark.sql import functions as F

# Group by order_id and count the distinct event_time values
inconsistent_records_count = udf4.groupby("order_id").agg(F.countDistinct("event_time").alias("unique_event_times"))

# Filter for records where the count of unique event_time values is greater than 1
inconsistent_records_count = inconsistent_records_count.filter(inconsistent_records_count["unique_event_times"] > 1)

# Count the number of inconsistent records
num_inconsistent_records = inconsistent_records_count.count()

# Display the number of inconsistent records
print("Number of inconsistent records:", num_inconsistent_records)


Number of inconsistent records: 0


# Download file after removing inconsistent records

In [49]:
output_path = "udf4_file"

udf4.coalesce(1).write.csv(output_path, header=True)


# Counf unique Order IDs (after removing inconsistent records)

In [51]:
from pyspark.sql.functions import countDistinct

# Count the number of unique user_id values
uc = udf4.select(countDistinct("order_id")).collect()[0][0]

# Display the count of unique user_id values
print("Number of unique order_id values:", uc)


24/02/14 22:55:51 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/02/14 22:55:51 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


Number of unique order_id values: 1176381


# Counf unique Event time (after removing inconsistent records)

In [50]:
from pyspark.sql.functions import countDistinct

# Count the number of unique user_id values
unique_user_count = udf4.select(countDistinct("event_time")).collect()[0][0]

# Display the count of unique user_id values
print("Number of unique event_time values:", unique_user_count)


24/02/14 22:55:32 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/02/14 22:55:32 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/02/14 22:55:32 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/02/14 22:55:32 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


Number of unique event_time values: 1176381


# Data cleaning after Phase 1

In [2]:
!pip install findspark
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("MySparkApp").getOrCreate()


Defaulting to user installation because normal site-packages is not writeable


24/02/15 15:05:51 WARN Utils: Your hostname, tejasshinde-Nitro-AN515-55 resolves to a loopback address: 127.0.1.1; using 192.168.1.155 instead (on interface wlp0s20f3)
24/02/15 15:05:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/15 15:05:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Upload partial clean data

In [3]:
data_path = "other_clean.csv"
ocdf = spark.read.csv(data_path, header=True, inferSchema=True)


# Count total no. of records

In [4]:
rec_cnt = ocdf.count()
print(rec_cnt)


2088263


# check null values

In [5]:
from pyspark.sql import functions as F
null_counts = [F.sum(F.col(column).isNull().cast("integer")).alias(column) for column in ocdf.columns]

# Describe the DataFrame with null value counts
null_counts_df = ocdf.agg(*null_counts)

null_counts_df.show()

+----------+--------+----------+-----------+-------------+-----+-----+-------+
|event_time|order_id|product_id|category_id|category_code|brand|price|user_id|
+----------+--------+----------+-----------+-------------+-----+-----+-------+
|         0|       0|         0|          0|            0|    0|    0|1551952|
+----------+--------+----------+-----------+-------------+-----+-----+-------+



# Drop null users so that users can be analyzed

In [6]:

spark = SparkSession.builder \
    .appName("RemoveNullUserIDs") \
    .getOrCreate()

drpusr = ocdf.dropna(subset=["user_id"])


24/02/15 12:44:48 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [7]:
rec_cnt = drpusr.count()
print(rec_cnt)

536311


# Download file after removing null users

In [1]:
output_path = "Phase1/drop_user"

drpusr.coalesce(1).write.csv(output_path, header=True)


NameError: name 'drpusr' is not defined

# Load drop_user file

In [24]:
data_path = "Phase1/drop_user.csv"
drpusr = spark.read.csv(data_path, header=True, inferSchema=True)


# Count unique users

In [5]:
from pyspark.sql.functions import countDistinct

# Count the number of unique user_id values
uc = drpusr.select(countDistinct("user_id")).collect()[0][0]

# Display the count of unique user_id values
print("Number of unique user values:", uc)


Number of unique user values: 229228


# Count unique orders

In [10]:
from pyspark.sql.functions import countDistinct

# Count the number of unique user_id values
uc = drpusr.select(countDistinct("order_id")).collect()[0][0]

# Display the count of unique user_id values
print("Number of unique order IDs values:", uc)


Number of unique order IDs values: 389511


# Count unique event times

In [22]:
from pyspark.sql.functions import countDistinct

# Count the number of unique user_id values
uc = drpusr.select(countDistinct("event_time")).collect()[0][0]

# Display the count of unique user_id values
print("Number of unique event times:", uc)


Number of unique event times: 379353


# Calculate the total amount spent


In [ ]:
from pyspark.sql.functions import col, sum


spark = SparkSession.builder \
    .appName("TotalAmount") \
    .getOrCreate()

total_amount = drpusr.select(sum(col("price"))).collect()[0][0]

print("Total amount spent:", total_amount)




# Max spent by user

In [18]:
from pyspark.sql.functions import max

spark = SparkSession.builder \
    .appName("MaxAvgAmountSpent") \
    .getOrCreate()

max = drpusr.agg(max("price")).collect()[0][0]

print("Max spent:", max)



24/02/15 15:42:37 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Max spent: 11574.05


# Min and Mean spent 

In [20]:
from pyspark.sql.functions import min, mean

spark = SparkSession.builder \
    .appName("StatisticsAvgAmountSpent") \
    .getOrCreate()



# Find the minimum and mean values of the column avg_amount_spent
statistics = drpusr.agg(min("price"), mean("price")).collect()[0]

# Extract the minimum and mean values
min = statistics[0]
mean = statistics[1]

# Print the minimum and mean values
print("Min spent:", min)
print("Mean spent:", mean)



24/02/15 15:44:23 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Min spent: 0.0
Mean spent: 214.5357087772056


# Create new dataframe : Average amount spent by each user

In [7]:
from pyspark.sql.functions import avg

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("AverageAmountSpent") \
    .getOrCreate()

avgamt = drpusr.groupBy("user_id").agg(avg("price").alias("avg_amount_spent"))

avgamt.show()




+-------------------+------------------+
|            user_id|  avg_amount_spent|
+-------------------+------------------+
|1515915625484582166|           27.4675|
|1515915625484624152|              6.92|
|1515915625456673752|            254.61|
|1515915625456170778|195.07857142857142|
|1515915625464630612|154.53615384615384|
|1515915625448144030|            115.72|
|1515915625456384238|             74.05|
|1515915625450815072|             20.81|
|1515915625474388138|116.17999999999999|
|1515915625454286948|196.40571428571428|
|1515915625464215439|            601.83|
|1515915625484623565|333.80125000000004|
|1515915625456780260|147.08333333333331|
|1515915625484629937|             84.93|
|1515915625484623070|            297.43|
|1515915625472058435|             25.44|
|1515915625463925870|            624.98|
|1515915625459124510|           154.375|
|1515915625484654720|            25.555|
|1515915625484645344|           167.805|
+-------------------+------------------+
only showing top

In [17]:
from pyspark.sql.functions import col, sum


spark = SparkSession.builder \
    .appName("TotalAmount") \
    .getOrCreate()

total_amount = avgamt.select(sum(col("avg_amount_spent"))).collect()[0][0]

print("Total amount spent:", total_amount)




Total amount spent: 51361913.93574576


In [8]:
rc = avgamt.count()
print(rc)

229228


In [9]:
output_path = "Phase1/avg_amt_spent"

avgamt.coalesce(1).write.csv(output_path, header=True)


# Find the maximum value of the column avg_amount_spent


In [11]:
from pyspark.sql.functions import max

spark = SparkSession.builder \
    .appName("MaxAvgAmountSpent") \
    .getOrCreate()

max = avgamt.agg(max("avg_amount_spent")).collect()[0][0]

print("Max value of avg_amount_spent:", max)



Max value of avg_amount_spent: 9606.48


In [12]:
from pyspark.sql.functions import min, mean

spark = SparkSession.builder \
    .appName("StatisticsAvgAmountSpent") \
    .getOrCreate()



# Find the minimum and mean values of the column avg_amount_spent
statistics = avgamt.agg(min("avg_amount_spent"), mean("avg_amount_spent")).collect()[0]

# Extract the minimum and mean values
min = statistics[0]
mean = statistics[1]

# Print the minimum and mean values
print("Min value of avg_amount_spent:", min)
print("Mean value of avg_amount_spent:", mean)



24/02/15 15:25:56 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Min value of avg_amount_spent: 0.0
Mean value of avg_amount_spent: 224.0647474817464


# No. of users who spent 0

In [15]:

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("CountUsersSpentZero") \
    .getOrCreate()

# Count the number of users who spent 0
users_spent_zero = avgamt.filter(avgamt["avg_amount_spent"] == 0).select("user_id").distinct().count()

# Print the number of users who spent 0
print("Number of users who spent 0:", users_spent_zero)



Number of users who spent 0: 5


# Count distinct category in drop_user

In [26]:
from pyspark.sql.functions import split, col

# Extract the category before the first dot '.'
df_with_category = drpusr.withColumn("category_before_dot", split(col("category_code"), "\\.")[0])

# Count the distinct records before the first dot '.'
distinct_category_before_dot_count = df_with_category.select("category_before_dot").distinct().count()

# Display the count of distinct records before the first dot '.'
print("Number of distinct records before the first dot '.':", distinct_category_before_dot_count)


Number of distinct records before the first dot '.': 14


In [27]:
from pyspark.sql.functions import split, col

# Extract the category before the first dot '.'
df_with_category = drpusr.withColumn("category_before_dot", split(col("category_code"), "\\.")[0])

# Select and display the distinct names of records before the first dot '.'
distinct_category_before_dot_names = df_with_category.select("category_before_dot").distinct().collect()
distinct_category_before_dot_names = [row["category_before_dot"] for row in distinct_category_before_dot_names]

# Display the distinct names of records before the first dot '.'
print("Distinct names of records before the first dot '.' in the 'category_code' column:")
for name in distinct_category_before_dot_names:
    print(name)


Distinct names of records before the first dot '.' in the 'category_code' column:
medicine
computers
auto
stationery
sport
other
apparel
appliances
country_yard
furniture
accessories
kids
electronics
construction


# Load category data

In [81]:
data_path = "category.csv"
cat = spark.read.csv(data_path, header=True, inferSchema=True)


In [57]:
cat.show()

+-------------------+----------+----------+-----------+--------------------+-------+------+-------------------+-----------+
|         event_time|  order_id|product_id|category_id|       category_code|  brand| price|            user_id|   category|
+-------------------+----------+----------+-----------+--------------------+-------+------+-------------------+-----------+
|2020-04-29 20:11:49|2.29807E18|1.51597E18| 2.26811E18|computers.periphe...|     hp|152.52|1515915625443020000|  computers|
|2020-04-29 23:42:11|2.29818E18|1.51597E18| 2.26811E18|electronics.audio...|samsung|  8.08|1515915625445930000|electronics|
|2020-04-30 18:01:51|2.29873E18|1.51597E18| 2.36074E18|appliances.enviro...|   beko|231.46|1515915625446610000| appliances|
|2020-04-30 19:27:36|2.29877E18|1.51597E18| 2.26811E18|electronics.audio...|  razer|104.14|1515915625452720000|electronics|
|2020-04-30 21:37:48|2.29884E18|1.51597E18| 2.26811E18|electronics.audio...|philips| 32.38|1515915625452810000|electronics|
|2020-05

In [42]:
cat = cat.drop('category_before_dot')

In [43]:
cat.show()

+-------------------+----------+----------+-----------+--------------------+-------+------+-------------------+-----------+
|         event_time|  order_id|product_id|category_id|       category_code|  brand| price|            user_id|   category|
+-------------------+----------+----------+-----------+--------------------+-------+------+-------------------+-----------+
|2020-04-29 20:11:49|2.29807E18|1.51597E18| 2.26811E18|computers.periphe...|     hp|152.52|1515915625443020000|  computers|
|2020-04-29 23:42:11|2.29818E18|1.51597E18| 2.26811E18|electronics.audio...|samsung|  8.08|1515915625445930000|electronics|
|2020-04-30 18:01:51|2.29873E18|1.51597E18| 2.36074E18|appliances.enviro...|   beko|231.46|1515915625446610000| appliances|
|2020-04-30 19:27:36|2.29877E18|1.51597E18| 2.26811E18|electronics.audio...|  razer|104.14|1515915625452720000|electronics|
|2020-04-30 21:37:48|2.29884E18|1.51597E18| 2.26811E18|electronics.audio...|philips| 32.38|1515915625452810000|electronics|
|2020-05

In [ ]:
chagnge datatypes

In [44]:
cat.dtypes

[('event_time', 'timestamp'),
 ('order_id', 'double'),
 ('product_id', 'double'),
 ('category_id', 'double'),
 ('category_code', 'string'),
 ('brand', 'string'),
 ('price', 'double'),
 ('user_id', 'bigint'),
 ('category', 'string')]

In [46]:
cat1 = cat.withColumn("user_id", col("user_id").cast("string"))

In [47]:
cat1.dtypes

[('event_time', 'timestamp'),
 ('order_id', 'double'),
 ('product_id', 'double'),
 ('category_id', 'double'),
 ('category_code', 'string'),
 ('brand', 'string'),
 ('price', 'double'),
 ('user_id', 'string'),
 ('category', 'string')]

In [48]:
cat1 = cat1.withColumn("order_id", col("order_id").cast("string"))
cat1 = cat1.withColumn("product_id", col("product_id").cast("string"))
cat1 = cat1.withColumn("category_id", col("category_id").cast("string"))


In [49]:
cat1.dtypes

[('event_time', 'timestamp'),
 ('order_id', 'string'),
 ('product_id', 'string'),
 ('category_id', 'string'),
 ('category_code', 'string'),
 ('brand', 'string'),
 ('price', 'double'),
 ('user_id', 'string'),
 ('category', 'string')]

In [52]:
output_path = "Phase1/Cat1"

cat.coalesce(1).write.csv(output_path, header=True)


In [53]:
cat = cat1

In [59]:
cat.dtypes

[('event_time', 'timestamp'),
 ('order_id', 'double'),
 ('product_id', 'double'),
 ('category_id', 'double'),
 ('category_code', 'string'),
 ('brand', 'string'),
 ('price', 'double'),
 ('user_id', 'bigint'),
 ('category', 'string')]

In [58]:
cat.show()

+-------------------+----------+----------+-----------+--------------------+-------+------+-------------------+-----------+
|         event_time|  order_id|product_id|category_id|       category_code|  brand| price|            user_id|   category|
+-------------------+----------+----------+-----------+--------------------+-------+------+-------------------+-----------+
|2020-04-29 20:11:49|2.29807E18|1.51597E18| 2.26811E18|computers.periphe...|     hp|152.52|1515915625443020000|  computers|
|2020-04-29 23:42:11|2.29818E18|1.51597E18| 2.26811E18|electronics.audio...|samsung|  8.08|1515915625445930000|electronics|
|2020-04-30 18:01:51|2.29873E18|1.51597E18| 2.36074E18|appliances.enviro...|   beko|231.46|1515915625446610000| appliances|
|2020-04-30 19:27:36|2.29877E18|1.51597E18| 2.26811E18|electronics.audio...|  razer|104.14|1515915625452720000|electronics|
|2020-04-30 21:37:48|2.29884E18|1.51597E18| 2.26811E18|electronics.audio...|philips| 32.38|1515915625452810000|electronics|
|2020-05

In [60]:
cat.dtypes

[('event_time', 'timestamp'),
 ('order_id', 'double'),
 ('product_id', 'double'),
 ('category_id', 'double'),
 ('category_code', 'string'),
 ('brand', 'string'),
 ('price', 'double'),
 ('user_id', 'bigint'),
 ('category', 'string')]

In [89]:
cat = cat.withColumn("order_id", col("order_id").cast("bigint"))
cat = cat.withColumn("product_id", col("product_id").cast("bigint"))
cat = cat.withColumn("category_id", col("category_id").cast("bigint"))


In [119]:
cat.dtypes

[('event_time', 'timestamp'),
 ('order_id', 'bigint'),
 ('product_id', 'bigint'),
 ('category_id', 'bigint'),
 ('category_code', 'string'),
 ('brand', 'string'),
 ('price', 'double'),
 ('user_id', 'bigint')]

In [98]:
cat.dtypes

[('event_time', 'timestamp'),
 ('order_id', 'bigint'),
 ('product_id', 'bigint'),
 ('category_id', 'bigint'),
 ('category_code', 'string'),
 ('brand', 'string'),
 ('price', 'double'),
 ('user_id', 'bigint'),
 ('category', 'string')]

In [99]:
cat.show()

+-------------------+-------------------+-------------------+-------------------+--------------------+-------+------+-------------------+-----------+
|         event_time|           order_id|         product_id|        category_id|       category_code|  brand| price|            user_id|   category|
+-------------------+-------------------+-------------------+-------------------+--------------------+-------+------+-------------------+-----------+
|2020-04-29 20:11:49|2298070000000000000|1515970000000000000|2268110000000000000|computers.periphe...|     hp|152.52|1515915625443020000|  computers|
|2020-04-29 23:42:11|2298180000000000000|1515970000000000000|2268110000000000000|electronics.audio...|samsung|  8.08|1515915625445930000|electronics|
|2020-04-30 18:01:51|2298730000000000000|1515970000000000000|2360740000000000000|appliances.enviro...|   beko|231.46|1515915625446610000| appliances|
|2020-04-30 19:27:36|2298770000000000000|1515970000000000000|2268110000000000000|electronics.audio..

In [64]:
output_path = "Phase1/Cat"

cat.coalesce(1).write.csv(output_path, header=True)


In [118]:
print(cat.count())

536311


In [117]:
from pyspark.sql import functions as F
null_counts = [F.sum(F.col(column).isNull().cast("integer")).alias(column) for column in cat.columns]

# Describe the DataFrame with null value counts
null_counts_df = cat.agg(*null_counts)

null_counts_df.show()

+----------+--------+----------+-----------+-------------+-----+-----+-------+
|event_time|order_id|product_id|category_id|category_code|brand|price|user_id|
+----------+--------+----------+-----------+-------------+-----+-----+-------+
|         0|       0|         0|          0|            0|    0|    0|      0|
+----------+--------+----------+-----------+-------------+-----+-----+-------+



In [127]:
from pyspark.sql.functions import countDistinct

# Count the number of unique user_id values
uc = cat.select(countDistinct("event_time")).collect()[0][0]

# Display the count of unique user_id values
print("Number of unique event times:", uc)


Number of unique event times: 379353


In [112]:
from pyspark.sql.functions import countDistinct

# Count the number of unique user_id values
uc = cat.select(countDistinct("order_id")).collect()[0][0]

# Display the count of unique user_id values
print("Number of unique order IDs:", uc)


Number of unique order IDs: 389511


In [126]:
from pyspark.sql.functions import countDistinct

# Count the number of unique user_id values
uc = cat.select(countDistinct("order_id")).collect()[0][0]

# Display the count of unique user_id values
print("Number of unique order IDs:", uc)


Number of unique order IDs: 389511


In [125]:
from pyspark.sql.functions import countDistinct

# Count the number of unique user_id values
uc = cat.select(countDistinct("user_id")).collect()[0][0]

# Display the count of unique user_id values
print("Number of unique user IDs:", uc)


Number of unique user IDs: 229228


In [124]:
from pyspark.sql.functions import countDistinct

# Count the number of unique user_id values
uc = cat.select(countDistinct("category")).collect()[0][0]

# Display the count of unique user_id values
print("Number of unique order IDs:", uc)


Number of unique order IDs: 14


In [123]:
from pyspark.sql.functions import countDistinct

# Count the number of unique user_id values
uc = cat.select(countDistinct("category_code")).collect()[0][0]

# Display the count of unique user_id values
print("Number of unique order IDs:", uc)


Number of unique order IDs: 124


In [122]:
data_path = "category_FINAL.csv"
cat = spark.read.csv(data_path, header=True, inferSchema=True)


In [134]:
from pyspark.sql.functions import countDistinct

# Count the number of unique user_id values
uc = cat.select(countDistinct("category")).collect()[0][0]

# Display the count of unique user_id values
print("Number of unique order IDs:", uc)


Number of unique order IDs: 14


In [128]:
cat.show()

+-------------------+-------------------+-------------------+-------------------+--------------------+-------+------+-------------------+-----------+
|         event_time|           order_id|         product_id|        category_id|       category_code|  brand| price|            user_id|   category|
+-------------------+-------------------+-------------------+-------------------+--------------------+-------+------+-------------------+-----------+
|2020-04-29 20:11:49|2298069964415828136|1515966223509122874|2268105407933187062|computers.periphe...|     hp|152.52|1515915625443027224|  computers|
|2020-04-29 23:42:11|2298175846491357353|1515966223509122666|2268105430162997728|electronics.audio...|samsung|  8.08|1515915625445938216|electronics|
|2020-04-30 18:01:51|2298729326712980173|1515966223509089265|2360741866917331945|appliances.enviro...|   beko|231.46|1515915625446617606| appliances|
|2020-04-30 19:27:36|2298772487720140990|1515966223509335414|2268105430162997728|electronics.audio..

In [129]:
cat.count()

536311

In [130]:
from pyspark.sql.functions import countDistinct

# Count the number of unique user_id values
uc = cat.select(countDistinct("product_id")).collect()[0][0]

# Display the count of unique user_id values
print("Number of unique order IDs:", uc)


Number of unique order IDs: 19078
